## 1.2 UM-Bridge Model Integration

In this subsection we use the MC simulation for estimating an integral. First we give the implementation of the considered model, which is the one dimensional function $f: \mathbb{R} \rightarrow \mathbb{R}$, where $f(x) = \sin(2\pi x)$.

In [ ]:
# umbridge model (server)
import nest_asyncio
nest_asyncio.apply()
import numpy as np
import umbridge
import time
import os

class Testmodel(umbridge.Model):# Define the 1D model function f(x)=sin(2*pi*x)

    def __init__(self):
        super().__init__("forward")

    def get_input_sizes(self, config):
        return [1]

    def get_output_sizes(self, config):
        return [1]

    def __call__(self, parameters, config):
        # Sleep for number of milliseconds defined in env var
        time.sleep(int(os.getenv("TEST_DELAY", 0)) / 1000)

        posterior = np.sin(2 * np.pi * parameters[0][0])
        return [[posterior]]

    def supports_evaluate(self):
        return True

testmodel = Testmodel()

umbridge.serve_models([testmodel], 4242) # start model server

======== Running on http://0.0.0.0:4242 ========
(Press CTRL+C to quit)


We are interested in the value of the integral $\int_{0}^{1} f(x) \textit{d}x$. We know by simple analysis calculations that this equals $0$. For the MC simulation we consider $N$ uniform distributed random variables $X_i \sim \mathscr{U}([0,1])$, $i=1,...,N$. Then we calculate the MC estimator
\begin{equation}
\hat{F}^{MC} = \frac{1}{N} \sum_{i=1}^{N} f(x_i)
\end{equation}
where $x_i$ are realizations of the random variables $X_i$. We note that $\mathbb{E}[f(X_i)] = \int_{0}^{1} f(x) \textit{d}x$. Thus, we obtain an unbiased estimator for the integral

Now we can make client requests using `MC_1.3`.